# This notebook processes CAFE v3 atmospheric daily data for building climatologies. Only the last 100 years are used.
Currently only runs on Raijin, as control run data not yet transferred to Canberra

In [1]:
# Import packages -----
import pandas as pd
import xarray as xr
import numpy as np
from ipywidgets import FloatProgress
from dateutil.relativedelta import relativedelta

#### Initialise

In [2]:
# Standard naming -----
fields = pd.DataFrame( \
        {'name_CAFE': ['lwflx', 'shflx', 't_ref', 'q_ref', 'u_ref', 'v_ref', 't_surf', 'h500', 
                       'precip', 'lwdn_sfc', 'olr', 'swdn_sfc', 'swup_toa'],
         'name_std' : ['lwf',   'shf',   't_ref', 'q_ref', 'u_ref', 'v_ref', 't_s',    'h500', 
                       'precip', 'lwf_dn_s', 'olr', 'swf_dn_s', 'swf_up_toa']}
                     )
name_dict = fields.set_index('name_CAFE').to_dict()['name_std']

fields

,name_CAFE,name_std
0,lwflx,lwf
1,shflx,shf
2,t_ref,t_ref
3,q_ref,q_ref
4,u_ref,u_ref
5,v_ref,v_ref
6,t_surf,t_s
7,h500,h500
8,precip,precip
9,lwdn_sfc,lwf_dn_s


#### Only use last 100 years

In [3]:
# Loop over all paths -----
base = '/g/data1/v14/coupled_model/v3/OUTPUT/'
years = range(400,500)

paths = []
for year in years:
    path = base + 'atmos_daily_0' + str(year) + '_01_01.nc'
    paths.append(path)

ds = xr.open_mfdataset(paths, autoclose=True) \
       .drop(['average_T1','average_T2','average_DT','time_bounds']) \
       .rename(name_dict)
        
if 'latb' in ds.dims:
    ds = ds.rename({'latb':'lat_2','lonb':'lon_2'})

/g/data3/hh5/public/apps/miniconda3/envs/analysis3/lib/python3.6/site-packages/xarray/conventions.py:416: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using dummy netCDF4.datetime objects instead, reason: dates out of range
  result = decode_cf_datetime(example_value, units, calendar)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3/lib/python3.6/site-packages/xarray/conventions.py:435: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using dummy netCDF4.datetime objects instead, reason: dates out of range
  calendar=self.calendar)


In [4]:
# Use year 2016 as time -----
path = '/g/data1/v14/forecast/v1/yr2016/mn1/OUTPUT.1/atmos_daily*.nc'
dataset = xr.open_mfdataset(path, autoclose=True)
time_use = xr.concat([dataset.time[:59], dataset.time[60:366]],dim='time')
time_ly = dataset.time[59]

In [5]:
# Make month_day array of month-day -----
m = [str(ds.time.values[i].timetuple()[1]).zfill(2) + '-' for i in range(len(ds.time))]
d = [str(ds.time.values[i].timetuple()[2]).zfill(2) for i in range(len(ds.time))]
md = np.core.defchararray.add(m, d)

# Replace time array with month_day array and groupby -----
ds['time'] = md
clim = ds.groupby('time').mean(dim='time',keep_attrs=True)
clim['time'] = time_use

In [6]:
# Replicate Feb 28th as Feb 29th to deal with leap years -----
clim_ly = clim.copy().sel(time='2016-02-28')
clim_ly['time'] = np.array([time_ly.values])
clim = xr.auto_combine([clim,clim_ly]).sortby('time')

In [8]:
clim

<xarray.Dataset>
Dimensions:     (lat: 90, lat_2: 91, lon: 144, lon_2: 145, nv: 2, time: 366)
Coordinates:
  * lon         (lon) float64 1.25 3.75 6.25 8.75 11.25 13.75 16.25 18.75 ...
  * lon_2       (lon_2) float64 0.0 2.5 5.0 7.5 10.0 12.5 15.0 17.5 20.0 ...
  * lat         (lat) float64 -89.49 -87.98 -85.96 -83.93 -81.91 -79.89 ...
  * lat_2       (lat_2) float64 -90.0 -88.99 -86.97 -84.94 -82.92 -80.9 ...
  * nv          (nv) float64 1.0 2.0
  * time        (time) datetime64[ns] 2016-01-01T12:00:00 ...
Data variables:
    lwf         (time, lat, lon) float64 dask.array<shape=(366, 90, 144), chunksize=(366, 90, 144)>
    shf         (time, lat, lon) float64 dask.array<shape=(366, 90, 144), chunksize=(366, 90, 144)>
    t_ref       (time, lat, lon) float64 dask.array<shape=(366, 90, 144), chunksize=(366, 90, 144)>
    q_ref       (time, lat, lon) float64 dask.array<shape=(366, 90, 144), chunksize=(366, 90, 144)>
    u_ref       (time, lat, lon) float64 dask.array<shape=(366, 90, 144

The history saving thread hit an unexpected error (OperationalError('unable to open database file',)).History will not be written to the database.


In [7]:
# Save the climatology -----
save_fldr = '/OSM/CBR/OA_DCFP/data/intermediate_products/pylatte_climatologies/'
clim.to_netcdf('cafe.c3.atmos.400_499.clim.nc', mode = 'w',
               encoding = {'time':{'dtype':'float','calendar':'JULIAN',
                           'units':'days since 0001-01-01 00:00:00'}})

RuntimeError: NetCDF: HDF error